In [ ]:
import papermill as pm
import os
from prefect.storage import Azure
import prefect
from prefect import task, Flow
import fsspec
from prefect.tasks.jupyter import ExecuteNotebook

In [ ]:
connection_string = os.environ.get("AZURE_STORAGE_CONNECTION_STRING")

In [ ]:
storage = Azure("prefect")
image = "carbonplan/cmip6-downscaling-prefect:latest"

In [ ]:
out_path = "az://cmip6"

In [ ]:
parameters = {"run_id": "gcm_name", "a": 3}

In [ ]:
import sys

In [ ]:
task = ExecuteNotebook(
    "papermill_test.ipynb",
    parameters=parameters,
    output_format="html",
    kernel_name="papermill-tutorial",
)

output = task.run()

In [ ]:
# out = fsspec.get_mapper('az://cmip6downscaling/scratch/papermill_test{}.ipynb'.format(parameters['run_id']),
#                           connection_string=connection_string)
# out = "abs://cmip6downscaling.blob.core.windows.net/cmip6/temp/papermill_test.ipynb"

In [ ]:
@task
def run_papermill_task(parameters):
    pm.execute_notebook(
        "papermill_test.ipynb",  # can't figure out how to write to azure. instead can write local, convert to html
        "analyses{}.ipynb".format(parameters["run_id"]),
        parameters=parameters,
    )

In [ ]:
with Flow(name=parameters["run_id"]) as flow:
    run_papermill_task(parameters)

In [ ]:
flow.run()

In [ ]:
# this might be what we want to do eventually but it struggles - AttributeError: 'OutStream' object has no attribute 'buffer'